In [6]:
import pandas as pd
import datetime
from os import listdir
import os, os.path
from os.path import isfile, join
import numpy as np
import csv
import re
import pandas as pd
import pm4py
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.log.exporter.xes import exporter as xes_exporter
from pm4py.objects.log.util import dataframe_utils
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
#log reading
df_categorias_autores=pd.read_csv("Log of analysis of time aspects.csv",index_col=0)
df_categorias_autores

,case:id,case:concept:name,concept:name,Variant,time:timestamp,Question,Report,Year,Category,Type,Tupla,Organization
0,A1 P2 BPI 2020,A1 P2 BPI 2020,Calculate cycle time,Calculate cycle time of the whole process for ...,2021-11-30 12:54:15.489743,P2,A1,2020,Academic,Analyze Time,A12020,"Technische Universität, Berlin"
1,A1 P2 BPI 2020,A1 P2 BPI 2020,Compare cycle time,Compare cycle time,2021-11-30 13:54:15.489743,P2,A1,2020,Academic,Analyze Time,A12020,"Technische Universität, Berlin"
2,A2 P2 BPI 2020,A2 P2 BPI 2020,Calculate cycle time,Get cycle time from the previous question,2021-11-30 14:54:15.489743,P2,A2,2020,Student,Analyze Time,A22020,"Faculty of Science, Utrecht University"
3,A2 P2 BPI 2020,A2 P2 BPI 2020,Compare cycle time,Compare cycle time,2021-11-30 15:54:15.489743,P2,A2,2020,Student,Analyze Time,A22020,"Faculty of Science, Utrecht University"
4,A7 P2 BPI 2020,A7 P2 BPI 2020,Calculate throughput,Calculate throughput,2021-11-30 16:54:15.489743,P2,A7,2020,Academic,Analyze Time,A72020,German Research Center for Artificial Intellig...
...,...,...,...,...,...,...,...,...,...,...,...,...
943,A19 P1 BPI 2020,A19 P1 BPI 2020,Group traces,Group traces by year,2022-01-08 19:54:15.489743,P1,A19,2020,Professional,Manipulation,A192020,Sberbank
944,A19 P1 BPI 2020,A19 P1 BPI 2020,Group traces,Group traces by activities,2022-01-08 20:54:15.489743,P1,A19,2020,Professional,Manipulation,A192020,Sberbank
945,A19 P1 BPI 2020,A19 P1 BPI 2020,Calculate statistics,Calculate Statistics of activities,2022-01-08 21:54:15.489743,P1,A19,2020,Professional,Calculate statistics,A192020,Sberbank
946,A19 P1 BPI 2020,A19 P1 BPI 2020,Calculate throughput,Calculate throughput,2022-01-08 22:54:15.489743,P1,A19,2020,Professional,Analyze Time,A192020,Sberbank


In [8]:
#identification of winners
winners=['A1-2015',"A2-2015","A13-2017","A18-2017","A22-2017","A4-2019","A12-2019","A1-2020","A2-2020"]
#creation of a column to identify winners easier
df_categorias_autores["repYear"]=[rep+"-"+str(year) for rep,year in zip(df_categorias_autores['Report'],df_categorias_autores['Year'])]

In [9]:
df_categorias_autores

,case:id,case:concept:name,concept:name,Variant,time:timestamp,Question,Report,Year,Category,Type,Tupla,Organization,repYear
0,A1 P2 BPI 2020,A1 P2 BPI 2020,Calculate cycle time,Calculate cycle time of the whole process for ...,2021-11-30 12:54:15.489743,P2,A1,2020,Academic,Analyze Time,A12020,"Technische Universität, Berlin",A1-2020
1,A1 P2 BPI 2020,A1 P2 BPI 2020,Compare cycle time,Compare cycle time,2021-11-30 13:54:15.489743,P2,A1,2020,Academic,Analyze Time,A12020,"Technische Universität, Berlin",A1-2020
2,A2 P2 BPI 2020,A2 P2 BPI 2020,Calculate cycle time,Get cycle time from the previous question,2021-11-30 14:54:15.489743,P2,A2,2020,Student,Analyze Time,A22020,"Faculty of Science, Utrecht University",A2-2020
3,A2 P2 BPI 2020,A2 P2 BPI 2020,Compare cycle time,Compare cycle time,2021-11-30 15:54:15.489743,P2,A2,2020,Student,Analyze Time,A22020,"Faculty of Science, Utrecht University",A2-2020
4,A7 P2 BPI 2020,A7 P2 BPI 2020,Calculate throughput,Calculate throughput,2021-11-30 16:54:15.489743,P2,A7,2020,Academic,Analyze Time,A72020,German Research Center for Artificial Intellig...,A7-2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,A19 P1 BPI 2020,A19 P1 BPI 2020,Group traces,Group traces by year,2022-01-08 19:54:15.489743,P1,A19,2020,Professional,Manipulation,A192020,Sberbank,A19-2020
944,A19 P1 BPI 2020,A19 P1 BPI 2020,Group traces,Group traces by activities,2022-01-08 20:54:15.489743,P1,A19,2020,Professional,Manipulation,A192020,Sberbank,A19-2020
945,A19 P1 BPI 2020,A19 P1 BPI 2020,Calculate statistics,Calculate Statistics of activities,2022-01-08 21:54:15.489743,P1,A19,2020,Professional,Calculate statistics,A192020,Sberbank,A19-2020
946,A19 P1 BPI 2020,A19 P1 BPI 2020,Calculate throughput,Calculate throughput,2022-01-08 22:54:15.489743,P1,A19,2020,Professional,Analyze Time,A192020,Sberbank,A19-2020


In [10]:
df_analisis = df_categorias_autores.groupby(by=['case:concept:name'])#group by case id, that means by each answer
df_agrupado = pd.DataFrame(columns=['case:concept:name','Operations', 'Year', 'Question',"repYear"])#definition of new dataframe
for name, group in df_analisis:

    year = group.iloc[0]['Year'] #year of the answer
    question = group.iloc[0]['Question']#question answered
    repYear=group.iloc[0]["Report"]+"-"+str(year)#year and report
    lista = group['concept:name'].to_list()#operations used in the answer
    #definition of new row
    new_row={'case:concept:name':name, 'Operations':lista, 'Year':year, 'Question':question,'repYear': repYear} 
    df_agrupado = df_agrupado.append(new_row, ignore_index=True)#addition of the new row
    
df_agrupado

,case:concept:name,Operations,Year,Question,repYear
0,A1 P1 BPI 2017,"[Calculate cycle time, Calculate cycle time, F...",2017,P1,A1-2017
1,A1 P1 BPI 2020,"[Filter traces, Calculate cycle time, Calculat...",2020,P1,A1-2020
2,A1 P2 BPI 2019,"[Group events by time, Calculate cycle time, R...",2019,P2,A1-2019
3,A1 P2 BPI 2020,"[Calculate cycle time, Compare cycle time]",2020,P2,A1-2020
4,A1 P5 BPI 2015,"[Calculate cycle time, Identify organizational...",2015,P5,A1-2015
...,...,...,...,...,...
121,A8 P1 BPI 2017,"[Filter traces, Calculate waiting time, Repres...",2017,P1,A8-2017
122,A8 P5 BPI 2015,"[Calculate cycle time, Compare cycle time, Gro...",2015,P5,A8-2015
123,A9 P1 BPI 2017,"[Calculate processing time, Calculate waiting ...",2017,P1,A9-2017
124,A9 P2 BPI 2019,"[Filter traces, Calculate cycle time, Identify...",2019,P2,A9-2019


In [11]:
#definition of winner and no winner for each answer
df_agrupado['Participant category']=["W" if case in winners else "NW" for case in df_agrupado['repYear']]

In [12]:
df_agrupado[(df_agrupado['Year']==2020) &(df_agrupado['Participant category']=="W")]

,case:concept:name,Operations,Year,Question,repYear,Participant category
1,A1 P1 BPI 2020,"[Filter traces, Calculate cycle time, Calculat...",2020,P1,A1-2020,W
3,A1 P2 BPI 2020,"[Calculate cycle time, Compare cycle time]",2020,P2,A1-2020,W
5,A1 P5 BPI 2020,"[Calculate cycle time, Find bottlenecks]",2020,P5,A1-2020,W
6,A1 P6 BPI 2020,"[Filter traces, Calculate cycle time]",2020,P6,A1-2020,W
55,A2 P1 BPI 2020,[Calculate cycle time],2020,P1,A2-2020,W
57,A2 P2 BPI 2020,"[Calculate cycle time, Compare cycle time]",2020,P2,A2-2020,W
58,A2 P4 BPI 2020,"[Filter activities, Group activities, Calculat...",2020,P4,A2-2020,W
60,A2 P5 BPI 2020,"[Calculate cycle time, Find bottlenecks, Repre...",2020,P5,A2-2020,W
61,A2 P6 BPI 2020,"[Calculate cycle time, Find bottlenecks, Repre...",2020,P6,A2-2020,W


In [13]:
#operations of the winners in 2019
df_categorias_autores[df_categorias_autores['case:concept:name'].isin(["A12 P2 BPI 2019","A4 P2 BPI 2019"])]['Variant'].value_counts()

Group traces depending on attributes                                                  6
Represent bar charts of cycle time                                                    4
Filter traces by attributes                                                           3
Calculate cycle time of the whole process for each subset of traces                   3
Calculate cycle time of a fragment of the process for a subset of traces              2
Calculate percentage of traces                                                        2
Calculate cycle time of only a subset of pairs of events for a subset of traces       2
Calculate Statistics of cycle time                                                    2
Calculate number of traces                                                            2
Group traces by activities                                                            1
Filter traces depending on the order of activities                                    1
Preprocess the traces of the log

In [14]:
#operations of all type of participants, where there is no clear difference
df_categorias_autores[(df_categorias_autores['Year']==2019) &  (df_categorias_autores['Question']=="P2")]['Variant'].value_counts()

Group traces depending on attributes                                                  15
Calculate number of traces                                                            11
Represent bar charts of cycle time                                                     9
Filter traces by activities                                                            8
Calculate percentage of traces                                                         7
Represent process map with cycle time                                                  7
Filter traces by attributes                                                            7
Represent histograms of cycle time                                                     6
Calculate cycle time of the whole process for each subset of traces                    6
Calculate cycle time of a fragment of the process for a subset of traces               6
Represent bar charts of attributes                                                     6
Identify attributes b

In [15]:
#definition of different similarity metrics
def sorensenIndex(list1,list2):
    set1=set(list1)
    set2=set(list2)
    intersection = set1.intersection(set2)
    len1=len(list(set1))
    len2=len(list(set2))
    sorensenIndex=float(2*len(list(intersection)))/(len1+len2)
    return sorensenIndex

In [16]:
def overlapIndex(list1,list2):
    set1=set(list1)
    set2=set(list2)
    intersection = set1.intersection(set2)
    len1=len(list(set1))
    len2=len(list(set2))
    overlapIndex=float(len(list(intersection)))/min([len1,len2])
    return overlapIndex

In [17]:
def jaccardIndex(list1,list2):
    set1=set(list1)
    set2=set(list2)
    intersection = set1.intersection(set2)
    tamIntersection=len(list(intersection))
    union = set1.union(set2)
    tamUnion=len(list(union))
    similitud=float(tamIntersection)/tamUnion
    return similitud

In [18]:
columns=['analisis','question', 'sorensen', 'jaccard','overlap','category']
df_metricas_preguntas = pd.DataFrame(columns=columns)
#comparison between all answers Winner vs Winner, No winner vs Winner, and No winner vs No winner (not grouped by question)
for i in range(len(df_agrupado)):
    for j in range(i+1,len(df_agrupado)):
            grupo = df_agrupado.iloc[i]['Operations']
            analisis =  df_agrupado.iloc[i]['case:concept:name'] + "-" + df_agrupado.iloc[j]['case:concept:name']
            question = name[1] + name[0]
            sorensen = sorensenIndex(grupo,df_agrupado.iloc[j]['Operations'])
            category = df_agrupado.iloc[i]['Participant category'] + "-" + df_agrupado.iloc[j]['Participant category']
            overlap = overlapIndex(grupo,df_agrupado.iloc[j]['Operations'])
            jaccard = jaccardIndex(grupo,df_agrupado.iloc[j]['Operations'])
            new_row={'analisis':analisis, 'question':question, 'sorensen':sorensen,'overlap':overlap,'jaccard':jaccard, 'category':category} 
            df_metricas_preguntas = df_metricas_preguntas.append(new_row, ignore_index=True)

In [19]:
df_metricas_preguntas['category'].unique()

array(['NW-W', 'NW-NW', 'W-NW', 'W-W'], dtype=object)

In [20]:
df_metricas_preguntas = df_metricas_preguntas.replace({'NW-W':'W-NW'})#rename of some comparisons that are the same type (NW-W and W-NW)

In [21]:
df_metricas_preguntas['category'].unique()

array(['W-NW', 'NW-NW', 'W-W'], dtype=object)

In [25]:
# mean results in each group:
df_metricas_preguntas.groupby("category").apply(lambda x: np.mean(x['sorensen']))

category
NW-NW    0.269757
W-NW     0.304903
W-W      0.347535
dtype: float64

In [26]:
df_metricas_preguntas.groupby("category").apply(lambda x: np.mean(x['jaccard']))

category
NW-NW    0.174335
W-NW     0.201671
W-W      0.233904
dtype: float64

In [24]:
df_metricas_preguntas.groupby("category").apply(lambda x: np.mean(x['overlap']))

category
NW-NW    0.415455
W-NW     0.468220
W-W      0.510556
dtype: float64

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------------

In [27]:
#Average number of operations in winners
df_winners=df_categorias_autores[df_categorias_autores["repYear"].isin(winners)]
avg_op_answer_winners=len(df_winners["concept:name"])/len(df_winners["case:concept:name"].unique())
print("Average number of operations in the answers of winners: "+ str(avg_op_answer_winners))

Average number of operations in the answers of winners: 6.0


In [28]:
#Average number of operations in no winners
df_no_winners=df_categorias_autores[~df_categorias_autores["repYear"].isin(winners)]
avg_op_answer_no_winners=len(df_no_winners["concept:name"])/len(df_no_winners["case:concept:name"].unique())
print("Average number of operations in the answers of no winners: "+ str(avg_op_answer_no_winners))

Average number of operations in the answers of no winners: 7.745454545454545


In [30]:
#number of operations of winners in each question and year
df_winners.groupby(["Question","Year"]).apply(lambda x: len(x)-1)

Question  Year
P1        2017    17
          2020     5
P2        2019    37
          2020     3
P4        2020     2
P5        2015    16
          2020     4
P6        2020     4
dtype: int64

In [31]:
df_winners["case:concept:name"].unique()

array(['A1 P2 BPI 2020', 'A2 P2 BPI 2020', 'A2 P4 BPI 2020',
       'A1 P5 BPI 2020', 'A2 P5 BPI 2020', 'A1 P6 BPI 2020',
       'A2 P6 BPI 2020', 'A2 P5 BPI 2015', 'A1 P1 BPI 2020',
       'A2 P1 BPI 2020', 'A13 P1 BPI 2017', 'A22 P1 BPI 2017',
       'A4 P2 BPI 2019', 'A1 P5 BPI 2015', 'A18 P1 BPI 2017',
       'A12 P2 BPI 2019'], dtype=object)

In [32]:
#average number of operations of winners per question
df_winners.groupby(['Question','Year']).apply(lambda x: len(x["concept:name"])/len(x["case:concept:name"].unique()))

Question  Year
P1        2017     6.0
          2020     3.0
P2        2019    19.0
          2020     2.0
P4        2020     3.0
P5        2015     8.5
          2020     2.5
P6        2020     2.5
dtype: float64

In [33]:
#average number of operations of no winners per question
df_no_winners.groupby(['Question','Year']).apply(lambda x: len(x["concept:name"])/len(x["case:concept:name"].unique()))

Question  Year
P1        2017    10.777778
          2020     7.235294
P2        2019    14.700000
          2020     3.812500
P4        2020     5.562500
P5        2015     9.857143
          2020     8.800000
P6        2020     3.363636
dtype: float64

In [39]:
#function to compare pair of answers
def comparePairsDF(df,func,func2,func3,qy):
    #func= function of sorensen
    #func2= function of jaccard
    #func3 = function of overlap
    
    parAnalysis=[]
    categories=[]
    indexSorensen=[]
    indexJaccard=[]
    indexOverlap=[]
    questionYear=[]
    
    idsAnalisis=df["repYear"].unique()
    cat1=""
    cat2=""
    #print(len(idsAnalisis))
    
    for i in range(len(idsAnalisis)-1):#for each analysis
        #print(idsAnalisis)
        #print("----------------------")
        ident=idsAnalisis[i] #get the id of an answer
        #print("i="+" "+str(i))
        operacionesIdent1=df[df["repYear"]==ident]['concept:name']#get the operation of answer
        #print(ident)
        #print("----------------------")
        #print("segunda lista")
        #print(idsAnalisis)
        
        if ident in winners:#check if it is a winner or not
            cat1="W"
        else:
            cat1="NW"
        j=0
        for j in range(i+1,len(idsAnalisis)):#repeat the same for the next answer
            ident2=idsAnalisis[j]
            #print("identificador dos")
            #print(ident+" "+ident2)

            operacionesIdent2=df[df["repYear"]==ident2]['concept:name']

            if ident2 in winners:
                cat2="W"
            else:
                cat2="NW"

            cats=cat1+"-"+cat2

            categories.append(cats)
            
            #get the indexes values between the operartions of the first and the second answer
            indexSorensen.append(func(operacionesIdent1,operacionesIdent2))
            indexJaccard.append(func2(operacionesIdent1,operacionesIdent2))
            indexOverlap.append(func3(operacionesIdent1,operacionesIdent2))
            parAnalysis.append(ident+"-"+ident2)#get which answers are being compared
            questionYear.append(qy)
    #add the results        
    dict_df={"parAnalisis":parAnalysis,"Categories":categories,"Jaccard index":indexJaccard,"Sorensen index":indexSorensen,"Overlap index":indexOverlap,"questionYear":questionYear}
    df_analisis=pd.DataFrame.from_dict(dict_df)
    #print(df_analisis)
    return df_analisis

In [40]:
agrupaciones_años_preguntas=df_categorias_autores.groupby(["Question","Year"])#grouping of answers by year

#definition of new dataframe which contains the comparisons between pairs of analysis that belong the same question
#"parAnalisis": case ids of the answers involved in the comparison
#Categories: types of participants involved in the comparison: Winner - Winner, No winner - Winner and No winner - No winner
dict_df_analysis={"parAnalisis":[],"Categories":[],"Jaccard index":[],"Sorensen index":[],"Overlap index":[],"questionYear":[]}
df_analisis_similarity=pd.DataFrame.from_dict(dict_df_analysis)

for name, df_group in agrupaciones_años_preguntas:
    #print(name)
    df_año_pregunta=comparePairsDF(df_group,sorensenIndex,jaccardIndex,overlapIndex,name)
    df_analisis_similarity=pd.concat([df_analisis_similarity,df_año_pregunta],ignore_index=True)

In [41]:
df_analisis_similarity

,parAnalisis,Categories,Jaccard index,Sorensen index,Overlap index,questionYear
0,A1-2017-A2-2017,NW-NW,0.000000,0.000000,0.000000,"(P1, 2017)"
1,A1-2017-A5-2017,NW-NW,0.222222,0.363636,0.400000,"(P1, 2017)"
2,A1-2017-A6-2017,NW-NW,0.384615,0.555556,0.833333,"(P1, 2017)"
3,A1-2017-A7-2017,NW-NW,0.272727,0.428571,0.500000,"(P1, 2017)"
4,A1-2017-A8-2017,NW-NW,0.111111,0.200000,0.250000,"(P1, 2017)"
...,...,...,...,...,...,...
981,A35-2020-A16-2020,NW-NW,0.000000,0.000000,0.000000,"(P6, 2020)"
982,A35-2020-A30-2020,NW-NW,0.250000,0.400000,0.500000,"(P6, 2020)"
983,A37-2020-A16-2020,NW-NW,0.166667,0.285714,0.500000,"(P6, 2020)"
984,A37-2020-A30-2020,NW-NW,0.142857,0.250000,0.333333,"(P6, 2020)"


In [42]:
df_analisis_similarity=df_analisis_similarity.reset_index()
df_analisis_similarity['Categories']=df_analisis_similarity['Categories'].replace("NW-W", "W-NW")#replace duplicate categories that are the same

In [43]:
df_analisis_similarity[(df_analisis_similarity["Categories"]=="W-W")]

,index,parAnalisis,Categories,Jaccard index,Sorensen index,Overlap index,questionYear
149,149,A13-2017-A22-2017,W-W,0.250000,0.400000,0.500000,"(P1, 2017)"
154,154,A13-2017-A18-2017,W-W,0.125000,0.222222,0.333333,"(P1, 2017)"
199,199,A22-2017-A18-2017,W-W,0.166667,0.285714,0.333333,"(P1, 2017)"
210,210,A1-2020-A2-2020,W-W,0.333333,0.500000,1.000000,"(P1, 2020)"
410,410,A4-2019-A12-2019,W-W,0.363636,0.533333,0.800000,"(P2, 2019)"
447,447,A1-2020-A2-2020,W-W,1.000000,1.000000,1.000000,"(P2, 2020)"
742,742,A2-2015-A1-2015,W-W,0.090909,0.166667,0.250000,"(P5, 2015)"
772,772,A1-2020-A2-2020,W-W,0.666667,0.800000,1.000000,"(P5, 2020)"
908,908,A1-2020-A2-2020,W-W,0.250000,0.400000,0.500000,"(P6, 2020)"


In [35]:
#---------------------------------------------------------------------------------------------------------------------------

In [41]:
#Average, variance and standard deviation of similiraties for each group and for each index
questionYearCategories=df_analisis_similarity.groupby("Categories").apply(lambda x: np.mean(x["Jaccard index"]) )
questionYearCategories

Categories
NW-NW    0.238942
W-NW     0.319126
W-W      0.360690
dtype: float64

In [42]:
questionYearCategoriesStd=df_analisis_similarity.groupby("Categories").apply(lambda x: x["Jaccard index"].std(ddof=0))
questionYearCategoriesStd

Categories
NW-NW    0.200258
W-NW     0.241761
W-W      0.277604
dtype: float64

In [43]:
questionYearCategoriesVar=df_analisis_similarity.groupby("Categories").apply(lambda x: x["Jaccard index"].var(ddof=0))
questionYearCategoriesVar

Categories
NW-NW    0.040103
W-NW     0.058448
W-W      0.077064
dtype: float64

In [44]:
#Sorensen
questionYearCategories=df_analisis_similarity.groupby("Categories").apply(lambda x: np.mean(x["Sorensen index"]))
questionYearCategories

Categories
NW-NW    0.347192
W-NW     0.438148
W-W      0.478660
dtype: float64

In [45]:
questionYearCategoriesStd=df_analisis_similarity.groupby("Categories").apply(lambda x: x["Sorensen index"].std(ddof=0))
questionYearCategoriesStd

Categories
NW-NW    0.243829
W-NW     0.256112
W-W      0.256299
dtype: float64

In [46]:
questionYearCategoriesVar=df_analisis_similarity.groupby("Categories").apply(lambda x: x["Sorensen index"].var(ddof=0))
questionYearCategoriesVar

Categories
NW-NW    0.059452
W-NW     0.065593
W-W      0.065689
dtype: float64

In [47]:
#Overlap index
questionYearCategories=df_analisis_similarity.groupby("Categories").apply(lambda x: np.mean(x["Overlap index"]) )
questionYearCategories

Categories
NW-NW    0.480633
W-NW     0.591833
W-W      0.635185
dtype: float64

In [48]:
questionYearCategoriesStd=df_analisis_similarity.groupby("Categories").apply(lambda x: x["Overlap index"].var(ddof=0))
questionYearCategoriesStd

Categories
NW-NW    0.111677
W-NW     0.104736
W-W      0.088176
dtype: float64

In [49]:
questionYearCategoriesStd=df_analisis_similarity.groupby("Categories").apply(lambda x: x["Overlap index"].std(ddof=0))
questionYearCategoriesStd

Categories
NW-NW    0.334181
W-NW     0.323630
W-W      0.296944
dtype: float64